This is a companion notebook for the book [Deep Learning with Python, Second Edition](https://www.manning.com/books/deep-learning-with-python-second-edition?a_aid=keras&a_bid=76564dff). For readability, it only contains runnable code blocks and section titles, and omits everything else in the book: text paragraphs, figures, and pseudocode.

**If you want to be able to follow what's going on, I recommend reading the notebook side by side with your copy of the book.**

This notebook was generated for TensorFlow 2.6.

## Generating images with variational autoencoders

### Sampling from latent spaces of images

### Concept vectors for image editing

### Variational autoencoders

### Implementing a VAE with Keras

**VAE encoder network**

In [ ]:
from tensorflow import keras # importing keras from tensorflow
from tensorflow.keras import layers # importing layers from tensorflow.keras

latent_dim = 2 # setting the latent dimension to 2 for the encoder model to be used in the VAE model below 

encoder_inputs = keras.Input(shape=(28, 28, 1)) # setting the input shape
x = layers.Conv2D(32, 3, activation="relu", strides=2, padding="same")(encoder_inputs) # setting the convolutional layer 
x = layers.Conv2D(64, 3, activation="relu", strides=2, padding="same")(x) # setting the convolutional layer
x = layers.Flatten()(x) # flattening the input
x = layers.Dense(16, activation="relu")(x) # setting the dense layer and activation function 
z_mean = layers.Dense(latent_dim, name="z_mean")(x) # setting the mean layer 
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x) # setting the log variance layer
encoder = keras.Model(encoder_inputs, [z_mean, z_log_var], name="encoder") # setting the encoder model

In [ ]:
encoder.summary() # printing the summary of the encoder model

**Latent-space-sampling layer**

In [ ]:
import tensorflow as tf # importing tensorflow

class Sampler(layers.Layer): # creating a class called Sampler
    def call(self, z_mean, z_log_var): # defining the call function with z_mean and z_log_var as arguments
        batch_size = tf.shape(z_mean)[0] # setting the batch size
        z_size = tf.shape(z_mean)[1] # setting the z size
        epsilon = tf.random.normal(shape=(batch_size, z_size)) # setting the epsilon value, which is a random normal distribution
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon # returning the mean and log variance

**VAE decoder network, mapping latent space points to images**

In [ ]:
latent_inputs = keras.Input(shape=(latent_dim,)) # setting the input shape
x = layers.Dense(7 * 7 * 64, activation="relu")(latent_inputs) # setting the dense layer and activation function
x = layers.Reshape((7, 7, 64))(x) # reshaping the input 
x = layers.Conv2DTranspose(64, 3, activation="relu", strides=2, padding="same")(x) # setting the convolutional layer
x = layers.Conv2DTranspose(32, 3, activation="relu", strides=2, padding="same")(x) # setting the convolutional layer
decoder_outputs = layers.Conv2D(1, 3, activation="sigmoid", padding="same")(x) # setting the convolutional layer
decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder") # setting the decoder model

In [ ]:
decoder.summary() # printing the summary of the decoder model

**VAE model with custom `train_step()`**

In [ ]:
class VAE(keras.Model): # creating a class called VAE
    def __init__(self, encoder, decoder, **kwargs): # defining the __init__ function with encoder and decoder as arguments
        super().__init__(**kwargs) # calling the super function
        self.encoder = encoder # setting the encoder
        self.decoder = decoder # setting the decoder
        self.sampler = Sampler() # setting the sampler
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss") # setting the total loss tracker
        self.reconstruction_loss_tracker = keras.metrics.Mean( # setting the reconstruction loss tracker
            name="reconstruction_loss") # setting the name of the reconstruction loss tracker 
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss") # setting the kl loss tracker

    @property # defining the property
    def metrics(self): # defining the metrics function
        return [self.total_loss_tracker, # returning the total loss tracker
                self.reconstruction_loss_tracker, # returning the reconstruction loss tracker
                self.kl_loss_tracker] # returning the kl loss tracker

    def train_step(self, data): # defining the train step function with data as an argument
        with tf.GradientTape() as tape: # setting the gradient tape
            z_mean, z_log_var = self.encoder(data) # setting the mean and log variance
            z = self.sampler(z_mean, z_log_var) # setting the sampler
            reconstruction = decoder(z) # setting the reconstruction
            reconstruction_loss = tf.reduce_mean( # setting the reconstruction loss
                tf.reduce_sum( # reducing the sum
                    keras.losses.binary_crossentropy(data, reconstruction), # calculating the binary cross entropy
                    axis=(1, 2) # setting the axis
                )
            )
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var)) # setting the kl loss as the negative of the kl divergence (this measures the difference between two probability distributions)
            total_loss = reconstruction_loss + tf.reduce_mean(kl_loss) # setting the total loss
        grads = tape.gradient(total_loss, self.trainable_weights) # calculating the gradients
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights)) # applying the gradients
        self.total_loss_tracker.update_state(total_loss) # updating the total loss tracker
        self.reconstruction_loss_tracker.update_state(reconstruction_loss) # updating the reconstruction loss tracker
        self.kl_loss_tracker.update_state(kl_loss) # updating the kl loss tracker
        return { # returning the total loss, reconstruction loss, and kl loss
            "total_loss": self.total_loss_tracker.result(), # returning the total loss
            "reconstruction_loss": self.reconstruction_loss_tracker.result(), # returning the reconstruction loss
            "kl_loss": self.kl_loss_tracker.result(), # returning the kl loss
        }

**Training the VAE**

In [ ]:
import numpy as np # importing numpy

(x_train, _), (x_test, _) = keras.datasets.mnist.load_data() # loading the mnist dataset into training and test data
mnist_digits = np.concatenate([x_train, x_test], axis=0) # concatenating the training and test data 
mnist_digits = np.expand_dims(mnist_digits, -1).astype("float32") / 255 # expanding the dimensions and normalizing the data

vae = VAE(encoder, decoder) # setting the VAE model with the encoder and decoder
vae.compile(optimizer=keras.optimizers.Adam(), run_eagerly=True) # compiling the VAE model with the Adam optimizer and running eagerly
vae.fit(mnist_digits, epochs=30, batch_size=128) # fitting the VAE model to the mnist digits for 30 epochs with a batch size of 128

**Sampling a grid of images from the 2D latent space**

In [ ]:
import matplotlib.pyplot as plt # importing matplotlib

n = 30 # setting n to 30
digit_size = 28 # setting the digit size to 28
figure = np.zeros((digit_size * n, digit_size * n)) # setting the figure to an array of zeros

grid_x = np.linspace(-1, 1, n) # setting the grid x values to a linear space from -1 to 1 with n values 
grid_y = np.linspace(-1, 1, n)[::-1] # setting the grid y values to a linear space from -1 to 1 with n values in reverse order

for i, yi in enumerate(grid_y): # iterating over the grid y values
    for j, xi in enumerate(grid_x): # iterating over the grid x values
        z_sample = np.array([[xi, yi]]) # setting the z sample
        x_decoded = vae.decoder.predict(z_sample) # decoding the z sample
        digit = x_decoded[0].reshape(digit_size, digit_size) # reshaping the digit
        figure[ # setting the figure
            i * digit_size : (i + 1) * digit_size, # setting the i digit size
            j * digit_size : (j + 1) * digit_size, # setting the j digit size
        ] = digit # setting the digit

plt.figure(figsize=(15, 15)) # setting the figure size to 15 by 15
start_range = digit_size // 2 # setting the start range to the digit size divided by 2
end_range = n * digit_size + start_range # setting the end range to n times the digit size plus the start range
pixel_range = np.arange(start_range, end_range, digit_size) # setting the pixel range
sample_range_x = np.round(grid_x, 1) # setting the sample range x
sample_range_y = np.round(grid_y, 1) # setting the sample range y
plt.xticks(pixel_range, sample_range_x) # setting the x ticks
plt.yticks(pixel_range, sample_range_y) # setting the y ticks
plt.xlabel("z[0]") # setting the x label
plt.ylabel("z[1]") # setting the y label
plt.axis("off") # turning off the axis
plt.imshow(figure, cmap="Greys_r") # showing the figure with the colormap Greys_r

### Wrapping up